In [1]:
# using config box you can access dictonary value diretly using (.)
d = {"key":"val", "key1":"val1"}

In [2]:
d['key']

'val'

In [5]:
d.key

AttributeError: 'dict' object has no attribute 'key'

In [3]:
from box import ConfigBox
d2 = ConfigBox(d)

In [4]:
d2.key

'val'

In [9]:
# use of ensure annotations 

def get_product(x:int, y:int) -> int:
    return x*y

In [10]:
get_product(2, 3)

6

In [12]:
# if you pass like this then our output is wrong
# in this case our fuction through an error put not for handling this we are 
# using one decorator called "@ensure_annotations"
get_product(2, '3')

'33'

In [14]:
from ensure import ensure_annotations

@ensure_annotations
def get_product(x:int, y:int) -> int:
    return x*y

In [15]:
get_product(2, 3)

6

In [16]:
# for this purpose we are using ensure_annotation decorator
get_product(2, '3')

EnsureError: Argument y of type <class 'str'> to <function get_product at 0x000002A47B774AF0> does not match annotation type <class 'int'>

## Loading data from Google Drive

In [17]:
# import gdown

# url = 'https://drive.google.com/file/d/1TKxQZecX1YxOrxLt2y-XFhuxW733M81-/view?usp=sharing'

# file_id = url.split("/")[-2]
# prefix = 'https://drive.google.com/uc?/export=download&id='
# gdown.download(prefix+file_id, "kidney-CT-Scan-data.zip")

Downloading...
From (uriginal): https://drive.google.com/uc?/export=download&id=1TKxQZecX1YxOrxLt2y-XFhuxW733M81-
From (redirected): https://drive.google.com/uc?/export=download&id=1TKxQZecX1YxOrxLt2y-XFhuxW733M81-&confirm=t&uuid=95fbcde9-eac6-470f-b81c-9cab9a586fdd
To: c:\Users\sumit\Desktop\Python_Dataset\Kidney-Disease-Classification-DVC-MLFlow\research\kidney-CT-Scan-data.zip
100%|██████████| 57.7M/57.7M [00:17<00:00, 3.37MB/s]


'kidney-CT-Scan-data.zip'

## Data Ingestion

In [1]:
import os

In [2]:
%pwd

'c:\\Users\\sumit\\Desktop\\Python_Dataset\\Kidney-Disease-Classification-DVC-MLFlow\\research'

In [3]:
os.chdir('../')

In [5]:

%pwd

'c:\\Users\\sumit\\Desktop\\Python_Dataset\\Kidney-Disease-Classification-DVC-MLFlow'

In [6]:
# Entity
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
from src.Stages.constants import *
from src.Stages.utils.common import read_yaml, create_directories

In [16]:
# Configrations
class ConfigurationManager:
    def __init__(self, config_filepath , params_filepath):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )
        
        return data_ingestion_config

In [17]:
import os
import zipfile
import gdown
from src.Stages.logging import logger
from src.Stages.utils.common import get_size

In [18]:
# Compoment
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''
        
        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [19]:
from pathlib import Path

CONFIG_FILE_PATH = Path("config/config.yaml")
PARAMS_FILE_PATH = Path("params.yaml")

In [21]:
# Pipeline
try:
    config = ConfigurationManager(CONFIG_FILE_PATH, PARAMS_FILE_PATH)
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-10-15 19:46:48,915: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-10-15 19:46:48,921: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-15 19:46:48,923: INFO: common: created directory at: artifacts]
[2023-10-15 19:46:48,928: INFO: common: created directory at: artifacts/data_ingestion]
[2023-10-15 19:46:48,933: INFO: 1168747554: Downloading data from https://drive.google.com/file/d/1TKxQZecX1YxOrxLt2y-XFhuxW733M81-/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (uriginal): https://drive.google.com/uc?/export=download&id=1TKxQZecX1YxOrxLt2y-XFhuxW733M81-
From (redirected): https://drive.google.com/uc?/export=download&id=1TKxQZecX1YxOrxLt2y-XFhuxW733M81-&confirm=t&uuid=b47edb8f-8c88-45db-b0af-3837ac85096d
To: c:\Users\sumit\Desktop\Python_Dataset\Kidney-Disease-Classification-DVC-MLFlow\artifacts\data_ingestion\data.zip
100%|██████████| 57.7M/57.7M [00:47<00:00, 1.22MB/s]

[2023-10-15 19:47:44,870: INFO: 1168747554: Downloaded data from https://drive.google.com/file/d/1TKxQZecX1YxOrxLt2y-XFhuxW733M81-/view?usp=sharing into file artifacts/data_ingestion/data.zip]
